The create_master_table function constructs a comprehensive master table by merging multiple data sources and features.

It starts by identifying fast-moving categories using the identify_fast_moving_category function, then processes weekly sales data with get_weekly_sales and create_primary_keys. It further identifies weekly sales through the identify_weekly_sales function and creates a target variable (sales_next_week) via create_target_variable. Additionally, it generates sales-related features like average sales, sales changes, and cumulative sales, and extracts time-related features, such as month, week of the year, and quarter, using get_sales_related_features and get_time_related_features. 

Finally, the function merges all these features with store information, fills missing values with zero, and returns the finalized master table. It also ensures clarity by renaming columns where necessary and handling potential conflicts in column names.

In [1]:
%run ./run_script.ipynb

conf = get_conf()

trans = get_datasources(conf)["trans_info"]
item = get_datasources(conf)["item_info"]
stores = get_datasources(conf)["outlets_info"]

trans = pre_process_transaction_info(trans)
item = pre_process_item_info(item)
store = pre_process_stores_info(stores)
no_categories = conf['params']["no_categories"]

In [2]:
def create_master_table(trans, item, no_categories, store):
    """
    Creating the master table
    
    Args:
        trans: Pandas DataFrame
            Transactions data
        item: Pandas DataFrame
            Items information
        no_categories: Number of item categories to be selected    
        store: Pandas DataFrame
            Store information
    
    Returns:
        Master Table: Pandas DataFrame
    """
    
    # Identify fast-moving categories
    fast_moving_category = identify_fast_moving_category(trans, item, no_categories)
    
    # Process the data for master table creation
    trans_weekly = get_weekly_sales(item, trans)
    primary_keys = create_primary_keys(trans_weekly)
    weekly_sales = identify_weekly_sales(trans_weekly, primary_keys)
    target_variable = create_target_variable(weekly_sales)
    sales_related_features = get_sales_related_features(target_variable)
    time_related_features = get_time_related_features(target_variable)

    # Merge data into the master table
    master_table = (
        primary_keys.merge(target_variable, on=["week", "outlet_code", "item_department"], how="left")
        .merge(sales_related_features, on=["week", "outlet_code", "item_department"], how="left")
        .merge(time_related_features, on=["week", "outlet_code", "item_department"], how="left")
        .merge(store, on=["outlet_code"], how="left")
        .merge(fast_moving_category, on=["outlet_code", "item_department"], how="left")  # Add fast_moving_category
        .fillna(0)
    )
    # If you want to prioritize specific versions, drop columns explicitly:
    master_table = master_table.drop(columns=['total_sales_qty_y'], errors='ignore')

    # Rename the retained column for clarity (optional)
    master_table.rename(columns={'total_sales_qty_x': 'total_sales_qty'}, inplace=True)
    
    return master_table

In [3]:
master_table=create_master_table(trans,item,no_categories,store)
master_table

,outlet_code,item_department,week,total_sales_qty,sales_next_week,fe_avg_4_week_sales,fe_4_weeks_std_dev_weekly,fe_4_weeks_weekly_min_sales,fe_4_weeks_weekly_max_sales,previous_week_sales,...,month,week_month,week_year,quarter_year,outlet_area,outlet_parking_lots,outlet_profile_category,outlet_cluster_category,item_category,row
0,A,Beverages,2022-01-17,598.0,1342.0,0.00,0.000000,0.0,0.0,0.0,...,1,3,3,1,10150,52,Moderate,Small,Ambient Liquid Milk,1.0
1,A,Beverages,2022-01-17,598.0,1342.0,0.00,0.000000,0.0,0.0,0.0,...,1,3,3,1,10150,52,Moderate,Small,Powdered Milk,2.0
2,A,Beverages,2022-01-17,598.0,1342.0,0.00,0.000000,0.0,0.0,0.0,...,1,3,3,1,10150,52,Moderate,Small,Single Consumption RTD Beverages,3.0
3,A,Beverages,2022-01-17,598.0,1342.0,0.00,0.000000,0.0,0.0,0.0,...,1,3,3,1,10150,52,Moderate,Small,Multiple Consumption RTD Beverages,4.0
4,A,Beverages,2022-01-17,598.0,1342.0,0.00,0.000000,0.0,0.0,0.0,...,1,3,3,1,10150,52,Moderate,Small,Non Flavored Tea,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,E,Grocery,2022-10-17,200.0,0.0,574.25,251.103398,200.0,737.0,737.0,...,10,3,42,4,10000,12,High,Medium,Ambient Instant Noodles,1.0
2996,E,Grocery,2022-10-17,200.0,0.0,574.25,251.103398,200.0,737.0,737.0,...,10,3,42,4,10000,12,High,Medium,Rice,2.0
2997,E,Grocery,2022-10-17,200.0,0.0,574.25,251.103398,200.0,737.0,737.0,...,10,3,42,4,10000,12,High,Medium,Sweet Biscuits & Cookies Regular,3.0
2998,E,Grocery,2022-10-17,200.0,0.0,574.25,251.103398,200.0,737.0,737.0,...,10,3,42,4,10000,12,High,Medium,Sugar Confectionary,4.0
